# Feature extraction with scikit-learn

> Written by Dr Daniel Buscombe, Northern Arizona University

> Part of a series of notebooks for image recognition and classification using deep convolutional neural networks

This notebook is an introduction to image feature extraction, which is usually a necessary step for machine learning based classifiers. This feature extraction is what deep learning tries to avoid

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Scikit-learn is a module for machine learning algorithms

* Supervised learning
* Unsupervised learning
* Dimensionality reduction
* Parameter selection
* Cross-validation

The scikit-learn toolbox (or sklearn) is a machine learning package built on the SciPy Stack, developed by an international community of practitioners under the leadership of a team of researchers in INRIA, France. 

It provides tools for regression, classification, clustering, dimensionality reduction, parameter selection and cross-validation. 

Gaussian mixture models, decision trees, support vector machines, and Gaussian processes are a few examples of the methods available to date.

## Fit/Predict

Scikit-learn's objects implements a fit/predict interface

* fit
    * learning step (supervised or unsupervised)
* predict
    * regression or classification
* The learned model can be stored using Python’s built-in persistence model, pickle

Sklearn is able to evaluate an estimator’s performance and parameters by cross-validation, optionally distributing the computation to several computer cores if necessary. The sklearn module implements machine learning algorithms as objects that provide a fit/predict interface. The fit method performs learning (supervised or unsupervised, according to the algorithm). The predict method performs regression or classification. The learned model can be saved for further usage by pickle, the Python’s built-in persistence model.

Let's load in a function to read an image

In [ ]:
from imageio import imread

## Feature engineering/extraction

![](figs/traditional-ml-deep-learning-2.png)

Before applying any ML method, you must first extract these features from your data: there is no formula for how to do this that applies across all domains, and thus this is where you as a data scientist must exercise your own intuition and expertise.

Feature selection is a process where you automatically select those features in your data that contribute most to the prediction variable or output in which you are interested.

Having irrelevant features in your data can decrease the accuracy of many models, especially linear algorithms like linear and logistic regression.

Three benefits of performing feature selection before modeling your data are:

* Reduces Overfitting: Less redundant data means less opportunity to make decisions based on noise.
* Improves Accuracy: Less misleading data means modeling accuracy improves.
* Reduces Training Time: Less data means that algorithms train faster.

We'll take a look at just two feature extraction methods, namely HOG (histogram of oriented gradients) and PCA (principal components analysis)

![](http://www.bogotobogo.com/python/scikit-learn/images/features/FeatureExtraction.png)

### HOG features

we will take a look at one such feature extraction technique, the Histogram of Oriented Gradients (HOG), which transforms image pixels into a vector representation that is sensitive to broadly informative image features regardless of confounding factors like illumination.

The Histogram of Gradients is a straightforward feature extraction procedure that was developed in the context of identifying pedestrians within images. HOG involves the following steps:

* Optionally pre-normalize images. This leads to features that resist dependence on variations in illumination.
* Convolve the image with two filters that are sensitive to horizontal and vertical brightness gradients. These capture edge, contour, and texture information.
* Subdivide the image into cells of a predetermined size, and compute a histogram of the gradient orientations within each cell.
* Normalize the histograms in each cell by comparing to the block of neighboring cells. This further suppresses the effect of illumination across the image.
* Construct a one-dimensional feature vector from the information in each cell.

I think of HOG features as image 'streamlines'

In [ ]:
## import libraries from scikit-image and call the hog function help
from skimage import feature, color
feature.hog?

In [ ]:
# import our S3 file system utility
import s3fs
fs = s3fs.S3FileSystem(anon=True)
root = 'esipfed/cdi-workshop'

Below we open an image from one of the S3 datasets (specifically, an airplane)

In [ ]:
#load in image
# the open file is called 'f'
with fs.open(root+'/imrecog_data/NWPU-RESISC45/test/airplane/airplane_700.jpg', 'rb') as f:
    image = color.rgb2gray(imread(f, 'jpg')) #'image' is the variable that contains the contents of 'f'

##extract HOG features
_, hog_vis = feature.hog(image, visualise=True)
# use most of the default values, but visualize = true, save visualization as 'hog_vis'
# the underscore is used because the feature.hog() method with visualize = T returns 2 objects, the first we are not interested in so its stored as '_'

##create a figure and show the input image and HOG features side by side 
fig, ax = plt.subplots(1, 2, figsize=(12, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
# we want 2 plots together, 1 x 2 array, store their indexes as 'ax'
ax[0].imshow(image, cmap='gray') # ax[0] means the first of two subplots
ax[0].set_title('input image')

im = ax[1].imshow(hog_vis) # ax[1] means the second of two subplots
plt.colorbar(im)
ax[1].set_title('visualization of HOG features');

## alternatively you could create the same plot with the following code:

#plt.figure(figsize=(12,6))
#plt.subplot(1,2,1)
#plt.imshow(image, cmap='gray')
#plt.title('Input Image')
#plt.axis('off')
#plt.subplot(1,2,2)
#plt.imshow(hog_vis)
#plt.title('Visualization of HOG features')
#plt.axis('off')

Playing with algorithm inputs ...

In [ ]:
##extract HOG features
# in this example we adjust some of the paramters from their default values, here we double the pixel size from 8x8 to 16x16
# use 30 oreiantations instead of 9, transform_sqrt = T
_, hog_vis = feature.hog(image, orientations=30, visualise=True, transform_sqrt=True, pixels_per_cell=(16, 16))

##create a figure and show the input image and HOG features side by side 
fig, ax = plt.subplots(1, 2, figsize=(12, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('input image')

ax[1].imshow(hog_vis)
ax[1].set_title('visualization of HOG features');

### Principal Components

PCA is fundamentally a dimensionality reduction algorithm, but it can also be useful as a tool for visualization, for noise filtering, for feature extraction and engineering, and much more.

PCA reduces a set of possibly correlated, high dimensional variables to a lower dimensional set of linearly correlated variables called principal components

This time we're treating each pixel as a unique feature, and we'll use PCA to reduce the number of features

First, let's set up a dataset to use. The EuroSAT data has images in 10 categories

In [ ]:
## list the subdirectories of the EuroSAT data, which will be our image 'classes'
cats = [f for f in fs.ls(root+'/imrecog_data/EuroSAT')]
cats

We'll import a function that will resize all images to a specified size

In [ ]:
from skimage.transform import resize

In [ ]:
images = []
files = [f for f in fs.ls(root+'/imrecog_data/UCMerced_LandUse/Images/test/freeway') if f.endswith('.jpg')]
for file in files:
    ##print('working on %s' % file) #uncomment this line if we want the image names printed to screen
    with fs.open(file, 'rb') as f:
        images.append(resize(color.rgb2gray(imread(f, 'jpg')), (128, 128)).flatten())

In [ ]:
## transform image data from list to numpy array
images = np.asarray(images)

In [ ]:
np.shape(images) ## the shape shows 34 images which are flattened into a vector 128 * 128 (16384) values long

In principal component analysis, the relationship between data dimensions is quantified by finding a list of the principal axes in the data, and using those axes to describe the dataset

In [ ]:
from sklearn.decomposition import PCA
PCA?

Using PCA for dimensionality reduction involves zeroing out one or more of the smallest principal components, resulting in a lower-dimensional projection of the data that preserves the maximal data variance.

### PCA demonstration

Here is an example of using PCA as a dimensionality reduction transform:

In [ ]:
# make PCA object with 1 component
pca  =  PCA(n_components =1)
## fit to data
pca.fit(images)
## apply the transformation
images_pca = pca.transform(images)
## compare shapes
print("original shape:   ", images.shape)
print("transformed shape:", images_pca.shape)

The transformed data has been reduced to a single dimension. Then we reproject the data back using the inverse transform

In [ ]:
images_new = pca.inverse_transform(images_pca)

print("transformed shape:", images_new.shape)

Let's take a look at the image vector and its associated reduced version 

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(images[0], 'k', label='Original')
plt.plot(images_new[0], 'r', label='Transformed')
plt.legend()
plt.xlabel('Pixel in image')
plt.ylabel('Pixel value')

To understand the effect of this dimensionality reduction, we can plot the inverse transform of this reduced data along with the original data:

In [ ]:
# first feature (pixel) versus first projected feature
plt.scatter(images[:, 0], images[:, 1], alpha=0.2)
plt.scatter(images_new[:, 0], images_new[:, 1], color='r', alpha=0.8)
plt.axis('equal');
plt.xlabel('Dimension 1 (first pixel across all 34 images)')
plt.ylabel('Dimension 2 (first pixel across all 34 projected images)')

The blue points are the original data, while the red points are the projected version. 

This makes clear what a PCA dimensionality reduction means: the information along the least important principal axis or axes is removed, leaving only the component(s) of the data with the highest variance. The fraction of variance that is cut out (proportional to the spread of points about the line formed in this figure) is roughly a measure of how much "information" is discarded in this reduction of dimensionality.

This reduced-dimension dataset is in some senses "good enough" to encode the most important relationships between the points: despite reducing the dimension of the data by 50%, the overall relationship between the data points are mostly preserved.

### Dimensionality reduction of airplane images

We're going to find 100 principal components using 
* Randomized SVD (singular value decomposition) solver
* Whiten (vectors are multiplied by the $\sqrt(N)$ and then divided by the singular values to ensure uncorrelated outputs)

In [ ]:
pca = PCA(svd_solver='randomized', n_components=100, whiten=True, random_state=42)
pca.fit(images)

The fit learns some quantities from the data, most importantly the "components" and "explained variance":

In [ ]:
print(pca.components_)

In [ ]:
print(pca.explained_variance_)

It can be interesting to visualize the images associated with the first several principal components (these components are known as "eigenvectors")

In [ ]:
fig, axes = plt.subplots(4, 6, figsize=(15, 15),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat): # enumerate creats pairs of values and indexes for those values, these values are...
    # stored as 'i' (index) and "ax' (subplot index)
    # axes.flat flattens the 4 x 6 grid of subplots into subplots 1 - 24, so they can be plotted sequentially
    ax.imshow(pca.components_[i].reshape(128, 128), cmap='gray')

A vital part of using PCA in practice is the ability to estimate how many components are needed to describe the data. This can be determined by looking at the cumulative explained variance ratio as a function of the number of components:

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

We see that about 20 components account for ~90% of the variance. That would lead us to believe that using these 20 components, we would recover most of the essential characteristics of the data. To make this more concrete, we can compare the input images with the images reconstructed from these 20 components:

In [ ]:
N = 20 #new number of components

pca = PCA(svd_solver='randomized', n_components=N, whiten=True, random_state=42)
pca.fit(images)

# Compute the components and projected images
components = pca.transform(images)
projected = pca.inverse_transform(components)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 4, figsize=(15, 15),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(4):
    ax[0, i].imshow(images[i].reshape(128, 128), cmap='gray')
    ax[1, i].imshow(projected[i].reshape(128, 128), cmap='gray')
    
ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel(str(N)+'-dim\nreconstruction');

The top row here shows the input images, while the bottom row shows the reconstruction of the images from just 50 of the initial features.

Although it reduces the dimensionality of the data, the projected images contain enough information that we might, by eye, recognize the individuals in the image. What this means is that our classification algorithm needs to be trained on 50-dimensional data rather than very high dimensional data, which depending on the particular algorithm we choose, can lead to a much more efficient classification.

## Exercises

1. How many principal components do you need for stationary camera time-series?

We have several time-series of images from a stationary camera in Grand Canyon. Such as this one:

In [ ]:
files = [f for f in fs.ls('cdi-workshop/semseg_data/RC1671L/flood4')]

In [ ]:
from scipy.misc import imresize

with fs.open(files[0], 'rb') as f:
    nx, ny = np.shape(imresize(color.rgb2gray(imread(f, 'jpg')), .25))

images = []
for file in files:
    with fs.open(file, 'rb') as f:
        images.append(imresize(color.rgb2gray(imread(f, 'jpg')), .25).flatten())

Make a plot showing all eight images

Decompose the image into principal components. How many components are needed to explain >85% of the variance?

Make a plot to visualize those components. What do you notice?

Transform the images into principal components, then do the inverse transform to get projected images

Make a plot comparing the original with projected images